In [ ]:
import string

exclude = set(string.punctuation)

def strip_punct(text):
    s = ''.join(ch for ch in text if ch not in exclude)
    return s



In [2]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import unquote

def Wikipedia_search(text,lang):
    url = "https://"+lang+".wikipedia.org/w/index.php?cirrusUserTesting=control&search="+text+"&title=Sp%C3%A9cial%3ARecherche&fulltext=1&ns0=1"
    
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data)
    try:
        tabledata = soup.find("ul", attrs={"class": "mw-search-results"})
        first_res = tabledata.findAll("li", attrs={"class": "mw-search-result"})[0]
        first_entity = first_res.find("a")["href"]
        first_entity = lang+first_entity
        first_entity = unquote(first_entity)
        return first_entity
    except AttributeError:
        return None
Wikipedia_search("INPI","fr")

'fr/wiki/INPI'

In [3]:
import spacy

In [4]:
import pickle
import pandas as pd

langs_dict = {"de":"de","ger":"de","fr":"fr","fre":"fr","it":"it","en":"en","sl":"sl","pl":"pl","pol":"pl","fi":"fi"}

with open('larger_selected_dataset.pickle', 'rb') as f:

#with open('selected_dataset.pickle', 'rb') as f:
    df = pickle.load(f)
    
df['langMaterial'] = df['langMaterial'].map(langs_dict)


In [5]:
df.head()

,id,langMaterial,unitTitle,titleProper,scopeContent,topic,filename
0,C122304194,fr,Usine Giraud,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Plan des ateliers de construction V. Giraud fi...,[economics],economics.json
1,C122304196,fr,Documents généraux.,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Historique par monsieur Hervé Arnoux (1993). C...,[economics],economics.json
2,C122304197,fr,Réparations et représentations pour les automo...,"119 J - Arnoux, fabrique de tracteurs, Miramas...","Garage Arnoux : vue de la façade sud, le long ...",[economics],economics.json
3,C122304198,fr,Motoculteurs Arnoux.,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Brevet d'invention pour un petit tracteur moto...,[economics],economics.json
4,C122304200,fr,Tracteurs Arnoux et leur outillage.,"119 J - Arnoux, fabrique de tracteurs, Miramas...",Tracteurs type VM 10 et VM 15 : prospectus et ...,[economics],economics.json


In [6]:
de_model = spacy.load("de_core_news_sm")
fr_model = spacy.load("fr_core_news_sm")
en_model = spacy.load("en_core_web_sm")
it_model = spacy.load("it_core_news_sm")

In [7]:
model_dict = {"fr":fr_model,"en":en_model,"de":de_model,"it":it_model}

In [8]:
topics = set([row["filename"] for index, row in df.iterrows()])

In [9]:
words_dict = {}

word_langs = {}

for index, row in df.iterrows():
    lang = row["langMaterial"]
    if lang in model_dict:
        model = model_dict[lang]
        text = row["unitTitle"] +" "+ row["titleProper"]+" "+ row["scopeContent"]
        topic = row["filename"]
        processed_text = model(text)
        # Analyze syntax
        words = [strip_punct(chunk.text) for chunk in processed_text.noun_chunks if len(chunk.text.split(" "))<4 and len(chunk.text)>2]

        for entity in processed_text.ents:
            e = strip_punct(entity.text)
            if len(e)>2:
                words.append(e)

        for word in words:
            #word = Wikipedia_search(word,lang)
            if word not in words_dict:
                words_dict[word] = {t:0.0 for t in topics}
                words_dict[word][topic] = 1
            else:
                words_dict[word][topic] += 1

            if word not in word_langs:
                word_langs[word] = {lang}
            else:
                word_langs[word].add(lang)


NameError: name 'ranking' is not defined

In [14]:
# this is simply for comparing across topics
ranking = {t:[] for t in topics}

for topic in topics:
    for word in words_dict.keys():
        if words_dict[word][topic]>0:
            score = words_dict[word][topic]
            #mean = [y for x,y in words_dict[word].items() if x!=topic]
            #mean = sum(mean)/len(mean)
            mean = 0
            score = words_dict[word][topic] - mean
            ranking[topic].append([word,score])

In [15]:
for topic,rank in ranking.items():
    print (topic)
    check = set()
    rank.sort(key=lambda x: x[1],reverse= True)
    for el in rank[:100]:
        l = list(word_langs[el[0]])[0]
        ent = Wikipedia_search(el[0],l)
        if ent!= None and ent not in check:
            print (ent)
            check.add(ent)
    print (" ")

catholicism.json
de/wiki/Monsignore
fr/wiki/Bureau_central_des_cultes
fr/wiki/Karim_Zaz
fr/wiki/Grade
fr/wiki/Produit_intérieur_brut
fr/wiki/Cabinet
fr/wiki/Jozef_De_Kesel
it/wiki/Jean
fr/wiki/Église_Saint-André_de_Fenain
it/wiki/France
fr/wiki/Louis_XII
fr/wiki/Henri_IV_(roi_de_France)
fr/wiki/Corse
de/wiki/Pierre
fr/wiki/François_Ier_(roi_de_France)
fr/wiki/Art_urbain
de/wiki/Etat
de/wiki/Elsass
fr/wiki/Bureau_central_de_renseignements_et_d'action
fr/wiki/Wildsteig
fr/wiki/René_Magritte
fr/wiki/Centre
fr/wiki/Bourse_de_Saint_Georges
fr/wiki/Monologue_intérieur
fr/wiki/Roger_Federer
fr/wiki/Moselle
fr/wiki/Conseil_technique_de_la_prévention_spécialisée
fr/wiki/Intérieur_(topologie)
fr/wiki/Maurice_(pays)
fr/wiki/Myers_Briggs_Type_Indicator
fr/wiki/Premier_ministre
fr/wiki/Archives_nationales_(France)
fr/wiki/Jean_M._Auel
fr/wiki/Louis-Joseph_Lebret
fr/wiki/Direction_générale_de_la_Sécurité_intérieure
fr/wiki/Marcel_Proust
de/wiki/Paris
fr/wiki/Qui
it/wiki/Joseph
fr/wiki/Art
fr/wiki/Lo